# SCA through Dino LFP data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/')
sys.path.append('/Users/ldliao/Research/Projects/spectralCA/notebooks_visualization/')
# sys.path.append('/Users/Lauren/Voytek/spectralCV')
# sys.path.append('/Users/rdgao/Documents/code/research/spectralCV')
# sys.path.append('/Users/rdgao/Documents/code/research/neurodsp/')
# sys.path.append('/Users/rdgao/Documents/code/research/fooof/')

In [35]:
%matplotlib inline

# imports
from sca_funcs import access_nt as asc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd

import neurodsp as ndsp
# from fooof import FOOOFGroup
from sca_funcs import sca

In [36]:
data_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_ephys.mat"
dino = io.loadmat(data_path, squeeze_me=True)

In [37]:
info_path = "/Users/ldliao/Research/Data/Dino/Dino_072114_mPFC_wakesleep.csv"
info = pd.read_csv(info_path,index_col='Unnamed: 0')

In [163]:
info

,Label,Start,Inter,End
0,REMEpisode,3752,NaN,3785
1,REMEpisode,4079,NaN,4130
2,REMEpisode,4822,NaN,4992
3,REM,3752,NaN,3785
4,REM,4079,NaN,4130
5,REM,4822,NaN,4897
6,REM,4903,NaN,4929
7,REM,4938,NaN,4992
8,SWSEpisode,3362,NaN,3751
9,SWSEpisode,3903,NaN,4069


In [55]:
set(info['Label'].values)

{'MA',
 'REM',
 'REMEpisode',
 'SWSEpisode',
 'SWSPacket',
 'Sleep',
 'Wake',
 'WakeInterruption',
 'WakeSleep'}

In [139]:
info;

In [38]:
info[-3:]

,Label,Start,Inter,End
175,Wake,5,NaN,3361
176,Sleep,3361,NaN,4992
177,WakeSleep,5,3361.0,4992


In [154]:
label = 'WakeSleep'

In [155]:
duration = list(zip(info[info['Label'] == label]['Start'].values, info[info['Label'] == label]['End'].values))

In [156]:
fs = 1250

In [157]:
chan_num = 4

In [158]:
# sleep
# data = dino['lfp'][:,1250*3361:1250*4992]
# start, end = wake_start, wake_end
# start_end = info[info['Label'] == label]['Start'].values, info[info['Label'] == label]['End'].values

# wake
data = []
for dur in duration:
    start, end = dur
    data.append(dino['lfp'][:,start*fs:end*fs])

In [159]:
data = np.column_stack(data)

In [160]:
data.shape

(4, 6233750)

In [161]:
fs = 1250
analysis_param = {'nperseg': 1250,
                 'noverlap': 0,
                 'spg_outlierpct': 2.,
                 'max_freq':200}

In [162]:
# initialize object with data and parameters
dino_sca = sca.SCA(analysis_param)
dino_sca.populate_ts_data(data, fs)

# compute all spectral attributes
dino_sca.compute_all_spectral()

# compute KS-test fit against null exponential hypothesis
dino_sca.compute_KS_expfit() # this takes a few minutes

# save out
dino_sca.save_spec_vars('../results/dino/'+label.lower(), save_spg=True)